# Textual similarity

We'll start by importing some things we will need later on.  Seaborn is a package that makes Matplotlib look nicer. 

In [ ]:
#pip install matplotlib seaborn nltk

In [ ]:
#pip install scikit-learn

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import os
import math
from matplotlib.colors import LogNorm
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
import operator

## Document vectors

We need a way to "vectorize" the text - i.e. convert each sample from a string of characters into some (fixed) number of features. We'll be doing this a lot, so we will go through it carefully.

To start, let's suppose we have a short list of documents:

In [ ]:
raw_texts = ["This is the text of the first document", 
             "This is the second one", 
             "Some other document", 
             "The next one", 
             "Another document - the last document in our list of documents"]

An easy way to quickly tokenize a string into a list of words is to use nltk's word_tokenize function.  For now we won't do any lemmatization.

In [ ]:
print(raw_texts[0])
print(word_tokenize(raw_texts[4]))

Since we're going to be treating documents as sequences of tokens, we can start by turning each document into a list of tokens:

In [ ]:
def text_to_documents(raw):
    docs = []
    for text in raw:
        docs.append([word.lower() for word in word_tokenize(text)])
    return docs

documents = text_to_documents(raw_texts)
documents

In order to do any vectorization, we're going to need to know what all the unique tokens are. For convenience, we define a function that takes a list of lists (i.e. a list of documents), and returns the unique tokens:

In [ ]:
def unique_terms(docs):
    words = {}
    for doc in docs:
        for word in doc:
            words[word] = True
    return list(words.keys())

E.g. the unique terms in just the *first* document:

In [ ]:
unique_terms([documents[0]])

Or, alternatively:

In [ ]:
set(documents[0])

What we're really interested in however is the list of unique terms across *all* documents:

In [ ]:
terms = unique_terms(documents)
print(terms)
len(terms)

Alternatively, we can create a list of sets:

In [ ]:
sets = [set(doc) for doc in documents]
sets

And then make a set that is the union of all of the sets in the list. This uses more advanced Python syntax (the asterisk is a list expansion):

In [ ]:
terms2 = set.union(*sets)
terms2

Now we have a list of terms, we can use this to vectorize a document. The order of information in our vectors will be determined by the order of our term list:

In [ ]:
def document_tf_vector(doc, terms):
    vector = []
    for term in terms:
        vector.append(doc.count(term))
    return vector

Let's try this first for a single document:

In [ ]:
print(documents[0])
document_tf_vector(documents[0], terms)

We can use this to turn all of our documents into TF vectors; if we turn this into a Pandas dataframe, we can also add our term list as column headings to better see what's going on:

In [ ]:
def document_tf_vectors(docs, terms):
    vectors = []
    for doc in docs:
        vectors.append(document_tf_vector(doc, terms))
    return vectors

In [ ]:
vectors = document_tf_vectors(documents, terms)
vectors

In [ ]:
df = pd.DataFrame(vectors, columns=terms)
df

Now we have vectors, we can start comparing them; let's define cosine similarity:

In [ ]:
def cosine_similarity(vector1, vector2):
    assert len(vector1) == len(vector2)
    product = 0
    norm1squared = 0
    norm2squared = 0
    for value1, value2 in zip(vector1, vector2):
        product += value1*value2
        norm1squared += value1**2
        norm2squared += value2**2
    return product/(math.sqrt(norm1squared * norm2squared))

In [ ]:
# How similar is vector 0 (document 0) to vector 1 (document 1)?
print(cosine_similarity(vectors[0], vectors[1]))

# How similar is vector 3 (document 3) to vector 3 (document 3)?
print(cosine_similarity(vectors[3], vectors[3]))

Although we're still using just term frequency, this is a good point to start looking at how we might make a global comparison among all of our documents.

In [ ]:
def compare_all_vectors(vectors):
    matrix = []
    for vector_i in vectors:
        row = []
        for vector_j in vectors:
            row.append(cosine_similarity(vector_i, vector_j))
        matrix.append(row)
    return matrix

In [ ]:
matrix = compare_all_vectors(vectors)
matrix

A matrix isn't that easy to read directly, but it corresponds to an intuitive heatmap - let's define a helper function for that too:

In [ ]:
def compare_all_heatmap(vectors):
    matrix = compare_all_vectors(vectors)
    plt.figure(figsize=(10,10))
    sns.heatmap(matrix, square=True, annot=True, cmap='Reds', cbar=True)
    plt.xlabel('Document vector #')
    plt.ylabel('Document vector #');
    plt.show()

In [ ]:
    compare_all_heatmap(vectors)

**Q:** Which two documents in this simple example are the most similar according to cosine similarity on TF vectors? With reference to the contents of the documents, *why* do we get this result?

Since we defined each individual step as a function, we can repeat things quite easily with a different corpus - let's use a very simple one with two terms:

In [ ]:
raw_texts = ["cat cat cat cat mouse", \
             "cat dog", \
             "dog dog cat dog", \
             "dog dog cat cat", \
             "cat cat cat dog", \
             "dog dog dog dog mouse" ]

In [ ]:
documents = text_to_documents(raw_texts)
terms = unique_terms(documents)
vectors = document_tf_vectors(documents, terms)
compare_all_heatmap(vectors)

**Q:**  Why are there two cells off the diagonal with one in them?

Now we can see similarity comparisons, we can implement Inverse Document Frequency weighting.

In [ ]:
def df(term, documents):
    count = 0
    for doc in documents:
        if term in doc:
            count += 1
    return count

In [ ]:
def idf(term, documents):
    n = len(documents)
    return math.log(n/df(term, documents))

We can now compute the document frequency and inverse document frequency of any term in our set of documents:

In [ ]:
print(df("dog", documents))
print(idf("dog", documents))

print(df("mouse", documents))
print(idf("mouse", documents))

We can use this to construct document vectors, this time using TF-IDF instead of just TF:

In [ ]:
def document_tfidf_vector(doc, terms, documents):
    vector = []
    for term in terms:
        vector.append(doc.count(term) * idf(term, documents))
    return vector

In [ ]:
def document_tfidf_vectors(docs, terms):
    vectors = []
    for doc in docs:
        vectors.append(document_tfidf_vector(doc, terms, docs))
    return vectors

Now we have this, we can construct a comparison matrix, and plot it as a heatmap exactly as before:

In [ ]:
vectors = document_tfidf_vectors(documents, terms)
compare_all_heatmap(vectors)

**Q:** Compare the output for this example (cats+dogs, TF-IDF) with the earlier output (cats+dogs, TF). Are there any differences? Explain why we get this result with this corpus (hint: think about the IDF values for each of the terms in this very small corpus).

**Q:** Our corpus was intentionally constructed with just two unique terms. What happens if you add a third term to one of the documents (e.g. mouse to the last one)?

**Q:** What happens if you additionally add another, different term to another of the documents (e.g. rat to the first one)? Which of the two charts (TF, or TF-IDF) is more radically altered by these changes? Why is this?  

**Q:** What if you add the first new term to the second document instead?

## A real corpus

So far we've just played around with very short documents to better understand what's going on. In this section, we'll load more realistic documents from text files instead.

**Note:** before running this code, you will need to unzip the text files and put the two folders in the same folder as this notebook.

So you should have a folder named `alice1` and another named `alice2`.

We define a helper function to read all the `.txt` files in a specified folder:

In [ ]:
def read_directory(folder):
    texts = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith(".txt"):
            labels.append(filename)
    labels.sort()
    for filename in labels:
        with open(os.path.join(folder, filename), encoding="utf-8") as f:
            texts.append(f.read())
    return [texts, labels]

Our first example text is a copy of Alice in Wonderland:

In [ ]:
raw_texts, labels = read_directory("alice1")
raw_texts[1][:1000]

Our function also gives us the filenames so we can keep track of which file is which document:

In [ ]:
labels

We defined all the steps above as functions, so we can re-run the same steps on our new corpus easily:

In [ ]:
documents = text_to_documents(raw_texts)
terms = unique_terms(documents)
vectors = document_tfidf_vectors(documents, terms)

In [ ]:
compare_all_heatmap(vectors)

Next, we'll load  a separate text - the manuscript version "Alice's adventures underground" ("alice2" folder), which differs substantially from the published 12 chapter version of the text ("alice1" folder):

In [ ]:
raw_texts2, labels2 = read_directory("alice2")
documents = text_to_documents(raw_texts + raw_texts2)

In [ ]:
labels2

In [ ]:
terms = unique_terms(documents)
vectors = document_tfidf_vectors(documents, terms)

In [ ]:
compare_all_heatmap(vectors)

We can see some interesting similarities between the two versions of Alice in Wonderland.

* Using the heatmap, which chapters of the 12-chapter edition (files in the "alice1" folder) likely correspond to the first chapter of the 4-chapter edition?

## N-grams

We will use the `ngrams` function from NLTK to extract n-grams from our texts.

In [ ]:
raw_texts = ["This is the text of the first document", \
             "This is the second one", \
             "Some other document", \
             "The next one", \
             "Another document - the last document in our list of documents"]
documents = text_to_documents(raw_texts)

For convenience, we'll define a variable "N" and use that as the "n" in "n-gram":

In [ ]:
N = 2

In [ ]:
ng = list(ngrams(documents[0], N))
ng

An easy way to define the Jaccard index will be to make use of Python's set functions. Unlike a list, a set does not allow duplicated members:

In [ ]:
a = set([1,2,2,2,2,3,4])
a

In [ ]:
b = set([2,3,4,5,5,5,5])
b

This is a convenient way to define Jaccard index, because Python has operators for intersection (&) and union (|) of sets:

In [ ]:
a&b

In [ ]:
a|b

By using these operators, our definition becomes very simple:

In [ ]:
def jaccard_index(a, b):
    a = set(a)
    b = set(b)
    if len(a|b) == 0:
        return 0
    else:
        return 1.0 * len(a&b)/len(a|b)

We can use this to compare the similarity of lists of ngrams, e.g.:

In [ ]:
ng1 = list(ngrams(documents[0], N))
ng2 = list(ngrams(documents[1], N))
jaccard_index(ng1, ng2)

For convenience, we can also define a function that builds a list of ngrams for each of our documents:

In [ ]:
def build_ngrams(documents, N):
    ngram_list = []
    for doc in documents:
        ngram_list.append(list(ngrams(doc, N)))
    return ngram_list

In [ ]:
ngram_list = build_ngrams(documents, N)
ngram_list

Just as with cosine similarity, a matrix containing all comparisons will help give us a good overview of all possible results:

In [ ]:
def compare_all_ngrams(ngram_list):
    matrix = []
    for ngram_i in ngram_list:
        row = []
        for ngram_j in ngram_list:
            row.append(jaccard_index(ngram_i, ngram_j))
        matrix.append(row)
    return matrix

In [ ]:
compare_all_ngrams(ngram_list)

And again, this matrix is going to be much easier to look at as a heatmap:

In [ ]:
def compare_all_ngrams_heatmap(ngram_list):
    matrix = compare_all_ngrams(ngram_list)
    plt.figure(figsize=(10,10))
    sns.heatmap(matrix, square=True, annot=True, cmap='Reds', cbar=True, norm=LogNorm())
    plt.xlabel('Document vector')
    plt.ylabel('Document vector');
    plt.show()


In [ ]:
compare_all_ngrams_heatmap(ngram_list)

We can use the set operators to examine which specific ngrams are shared between any two documents:

In [ ]:
set(ngram_list[0]) & set(ngram_list[1])

Although there are shared n-grams, there isn't a huge amount of obvious reuse in our current corpus.  Let's try the two versions of Alice. 

In [ ]:
N = 3
raw_texts, labels = read_directory("alice1")
raw_texts2, labels2 = read_directory("alice2")
documents = text_to_documents(raw_texts + raw_texts2)
ngram_list = build_ngrams(documents, N)
compare_all_ngrams_heatmap(ngram_list)

In [ ]:
print(len(ngram_list), len(ngram_list[0]))
ngram_list[0][:10]

**Q:** How dependent are the general trends visible in the heatmap on the chosen value of N? Experiment with different values.

## Sklearn implementation

Our implementation of TF-IDF and cosine similarity is very slow with large documents, because it's intentionally written to favour readability over performance. 

In the future, we will use the **much** more efficient implementation that is included in the sklearn library as "[TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)". You can experiment with this library, and compare the results with those from the code above.

As we will see, there are minor differences between the sklearn implementation and the standard version of TF-IDF described in class and implemented in this notebook. See the sklearn documentation for the differences.

### TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(norm=None,sublinear_tf=True)

In [ ]:
tfidf = vectorizer.fit_transform(raw_texts + raw_texts2)

In [ ]:
tfidf

In [ ]:
tfidf.toarray()

In [ ]:
compare_all_heatmap(tfidf.toarray())

### CountVectorizer (for ngrams)

In [ ]:
vectorizer = CountVectorizer(ngram_range=(2,2), lowercase=True)
counts = vectorizer.fit_transform(raw_texts + raw_texts2)

In [ ]:
print(len(vectorizer.get_feature_names_out()))
vectorizer.get_feature_names_out()[0:30]

In [ ]:
counts

We have counts of ngrams rather than lists of ngrams, so we use the original heatmap function, which uses cosine similarity.

In [ ]:
compare_all_heatmap(counts.toarray())